# Compute Spectra Workflow
- Identify material system of interest
- Specify simulation parameters
- Run simulation
- Extract simulation outputs

[this cell will be deleted / cleaned up in the future]

###  Import Dependencies

In [107]:
%matplotlib widget

#data tools
import pandas as pd
import numpy as np
import qgrid
from pymatgen.ext.matproj import MPRester
from pymatgen.vis import structure_chemview as viz

#simtool loading and interface
from simtool import findInstalledSimToolNotebooks,searchForSimTool
from simtool import getSimToolInputs,getSimToolOutputs,Run

#user interface utilities 
import os, stat
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output

### User Create ~/.mpkey.txt if it doesn't already exist

In [4]:
#key security
try:
    user = str(input('Paste MP API key: '))
    clear_output()
    if not user.isalnum():
        raise TypeError('Wrong Key')
    if user is None:
        raise TypeError('Empty')
    with open(os.path.expanduser('~/.mpkey.txt'), 'w') as keyfile:
        keyfile.write(user)
    os.chmod(os.path.expanduser('~/.mpkey.txt'), stat.S_IREAD | stat.S_IWRITE)
    del user
    print("Success")
except:
    print("Something seems wrong with your key")

Something seems wrong with your key


## User Prompted To pick their Semiconductor of Choice
### Choice (a): query MP and fliter on properties. Dataframe is updated with each selection.

In [108]:
# query MP, get all cubic systems
 
with open(os.path.expanduser("~/.mpkey.txt"), "r+") as file:
    apikey = file.readline()
rester = MPRester(apikey)
sc_dicts = rester.query({ "crystal_system": "cubic"},
                        ["task_id","pretty_formula","formula","elements","e_above_hull", "spacegroup.number", "band_gap", "crystal_system"])
sc_df = pd.DataFrame(sc_dicts)

# define a function for visualizing input structures

def mpid_plot(mpid):

    # import POSCAR file

    struct = rester.get_structure_by_material_id(mpid, final = False, conventional_unit_cell=True)
    POSCAR_str = struct.to(fmt = "poscar")

    lines = POSCAR_str.split('\n')

    # get the lattice information

    lattice = lines[1]
    cell_vectors = np.array([lines[2].split() , lines[3].split() , lines[4].split()]).astype(float)

    # get the list of sites

    sites = []
    for line in lines[8:]:
        if not line:
            break
        sites.append([line.split()[0],line.split()[1],line.split()[2]])

    # convert from fractional to xyz

    sites = np.array(sites).astype(float)
    xyz = np.matmul(sites,cell_vectors).transpose()

    # get the coordinates of the box

    corners = np.array([[0,1,1,0,0,0,0,1,1,0,0,1,1,1,1,0,0],[0,0,1,1,0,0,0,0,1,1,0,0,0,1,1,1,1],[0,0,0,0,0,1,1,1,1,1,1,1,0,0,1,1,0]]).T
    cell = np.matmul(corners,cell_vectors).transpose()
    
    box = pd.DataFrame({'x':cell[0],'y':cell[1],'z':cell[2]})

    # get a color dictionary

    elements = lines[5].split()
    atoms = lines[6].split()
    hues = ['tab:blue','tab:orange','tab:green','tab:red']
    colors = []
    for i, atom in enumerate(atoms):
        colors.extend([elements[i]]*int(atoms[i]))

    zip_iterator = zip(elements,hues)
    color_dict = dict(zip_iterator)
    
    size = 72
    POSCAR_df = pd.DataFrame({'x':xyz[0],'y':xyz[1],'z':xyz[2],'element':colors,'size':size})    
    fig = px.scatter_3d(POSCAR_df, x='x', y='y', z='z',
              color='element',labels={'x':'','y':'','z':''},size='size',size_max=size)
    
    box = px.line_3d(box,x='x', y='y', z='z')
    fig.add_traces(list(box.select_traces()))
    
    fig.update_layout(scene = dict(
                    xaxis = dict(
                        nticks=0,showbackground=False,showticklabels=False,),
                    yaxis = dict(
                        nticks=0,showbackground=False,showticklabels=False,),
                    zaxis = dict(
                        nticks=0,showbackground=False,showticklabels=False,),),
                    width=700,
                    margin=dict(r=10, l=10, b=10, t=10)
                  )

    fig.show()

In [109]:
# create widgets

mpid = widgets.IntText(
        value=2133,
        description='MPID:',
        disabled=False
)

plot_button = widgets.Button(
    description='plot',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='plot'
)

# TODO: figure out a system of taking input from EITHER mpid input, or qgrid selected data
        
def on_button_clicked(b):
    with output:
        try:
            selected_row = mpid_widget.get_changed_df().index[mpid_widget.get_selected_rows()][0]
            mpid_selected= sc_df.at[selected_row,'task_id']
            mpid.value = mpid_selected.split('-')[1]
        except:
            mpid_selected="mp-"+str(mpid.value)
        mpid_plot(mpid_selected)
        
        
def update_plot(args):
    mpid_selected="mp-"+str(mpid.value)
    mpid_plot(mpid_selected)
    
output = widgets.Output()
mpid.observe(update_plot,'value')  
plot_button.on_click(on_button_clicked)
display(mpid, plot_button, output)

# if you don't, search for your structure by filtering
mpid_widget = qgrid.show_grid(sc_df)

# filter by pretty_formula to look for a specific structure
display(mpid_widget)

IntText(value=2133, description='MPID:')

Button(description='plot', style=ButtonStyle(), tooltip='plot')

Output()

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

### Get structure from MP database
TODO: could be combined with the last cell

In [110]:
# This is passed to the simtool to perform simulations
# structure objects obtained directly from remote database queries must be passed to simtool as dictionaries to preserve maximum numerical precision
# quantum espresso is very sensitive to precision especially for relaxation calculations.
struct = rester.get_structure_by_material_id("mp-"+str(mpid.value), final = False, conventional_unit_cell=True)
struct_dict = struct.as_dict()#to(fmt="JSON")
struct_dict

{'@module': 'pymatgen.core.structure',
 '@class': 'Structure',
 'charge': None,
 'lattice': {'matrix': [[1.5920500003918698, -2.7575114888687695, 0.0],
   [1.5920500003918698, 2.7575114888687695, 0.0],
   [0.0, 0.0, 5.1551]],
  'a': 3.1841000007837397,
  'b': 3.1841000007837397,
  'c': 5.1551,
  'alpha': 90.0,
  'beta': 90.0,
  'gamma': 120.00000000000001,
  'volume': 45.26276950032406},
 'sites': [{'species': [{'element': 'Zn', 'occu': 1}],
   'abc': [0.6666666666666666, 0.3333333333333333, 0.5],
   'xyz': [1.5920500003918696, -0.9191704962895898, 2.57755],
   'label': 'Zn',
   'properties': {}},
  {'species': [{'element': 'Zn', 'occu': 1}],
   'abc': [0.3333333333333333, 0.6666666666666666, 0.0],
   'xyz': [1.5920500003918696, 0.9191704962895898, 0.0],
   'label': 'Zn',
   'properties': {}},
  {'species': [{'element': 'O', 'occu': 1}],
   'abc': [0.6666666666666666, 0.3333333333333333, 0.8803099999999999],
   'xyz': [1.5920500003918696, -0.9191704962895898, 4.538086080999999],
   'la

### Choice (b): upload your own poscar directly. No query necessary.

In [106]:
#users can pass their own structure as a poscar. the simtool will take the file contents as a dictionary which is produced here
struct = Structure.from_file("./POSCAR")
struct_dict = struct.as_dict()
struct_dict

NameError: name 'Structure' is not defined

# Perform structure relaxation, scf, and phonon computation and spectra extraction using simtool

### Find relax_sim simtool notebook and confirm

In [122]:
#simToolName = "670raman"
simToolName = "relax_sim"
simToolLocation = searchForSimTool(simToolName)
for key in simToolLocation.keys():
    print(f"{key} = {simToolLocation[key]}")

notebookPath = /home/nanohub/nykiel.4/opt_prop/opt_prop_suite/simtool/relax_sim.ipynb
simToolName = relax_sim
simToolRevision = None
published = False


In [123]:
installedSimToolNotebooks = findInstalledSimToolNotebooks(simToolName,returnString=True)
print(installedSimToolNotebooks)

{}


In [124]:
#Enter your values with units! The simtool will make sure you know what you're talking about.
inputs = getSimToolInputs(simToolLocation)
inputs

loglevel:
    type: Choice
    description: python logging module level setting
    options: ['DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL']

numnodes:
    type: Number
    description: Number of processors for mpi execution
    min: 1
    max: 8
    value: 1

walltime:
    type: Text
    description: Maximum time to wait for reduced job load on cluster in HH:MM:SS time format
    value: 01:00:00

pps:
    type: List
    description: List of pseudopotentials in UPF format to be used by simulator. Potentials must either exist in ./simtool/pseudo or be accessible at http://www.quantum-espresso.org/wp-content/uploads/upf_files/
    value: ['O.pbe-hgh.UPF', 'Zn.pbe-d-hgh.UPF']

smearing:
    type: Choice
    description: Setting the extent to which there is metal-like sharing of electrons
    options: ['smearing', 'fixed']

ecutwfc:
    type: Number
    description: kinetic energy cutoff for wavefunctions
    units: rydberg
    min: 25
    max: 400
    value: 120

ecutrho:
    type: Numb

In [19]:
# skip this: reimplementing widget definitions wip
class ToolSet():
    def __init__(self, inputs, struct_dict): #users_pp_file_list
        self.pp_list = []
        for filename in os.listdir("./simtool/pseudo/"):
            f = os.path.join("./simtool/pseudo/", filename)
            # get a list of all the PPs -- is this best instatiated here or globally?
            # if instanced here, the user could probably pass their own PPs to the constructor as well
            if os.path.isfile(f):
                self.pp_list.append(filename)

        # TODO: filter by selected compound compositions
        self.filtered_pp_list = self.pp_list

    def _create_widgets(self):
        self.log = widgets.Select(
            options=["DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"],
            value='DEBUG',
            # rows=10,
            description='Log Level:',
            disabled=False
        )
        self.walltime = widgets.Text(
            value='01:00:00',
            placeholder='walltime',
            description='walltime:',
            disabled=False
        )
        self.numnodes = widgets.IntText(
            value=8,
            placeholder='nodes',
            description='nodes:',
            disabled=False
        ) 
        self.button = widgets.Button(
            description='run simtool',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='run to submit qe simtool'
        )
        self.button.on_click(self._on_button_clicked)
        self.ecutwfc = widgets.BoundedFloatText(
            value=50,
            min=50,
            max=400,
            step=10,
            description='ecutwfc:',
            disabled=False
        )
        self.ecutrho = widgets.BoundedFloatText(
            value=200,
            min=200,
            max=1600,
            step=40,
            description='ecutrho:',
            disabled=False
        )
        self.smearing = widgets.Select(
            options=['smearing','fixed'],
            value='fixed',
            rows = 2,
            description='smearing:',
            disabled=False
        )
        self.pp_menu1 = widgets.Combobox(
            placeholder="choose a pseudopotential",
            options=filtered_pp_list,
            description='pseudopotential 1:',
            disabled=False
        )
        self.pp_menu2 = widgets.Combobox(
            placeholder="choose a pseudopotential",
            options=filtered_pp_list,
            description='pseudopotential 2:',
            disabled=False
        )
        self.output = widgets.Output()
        def _on_button_clicked(self, change):
            with output:
                print("submitting sim2l run with formula" , self.compound.value, self.spacegroup.value)
                runSim2l()
                r = Run(simToolLocation,inputs)

        # display(c, s, button, output)
        def runSim2l():
            inputs['loglevel'].value = log.value
            inputs['walltime'].value = walltime.value
            inputs['numnodes'].value = numnodes.value
            inputs['ecutwfc'].value = ecutwfc.value
            inputs['ecutrho'].value = ecutrho.value
            inputs['pps'].value = [pp_menu1.value, pp_menu2.value]
            inputs['smearing'].value = smearing.value
            inputs['struct_dict'].value = struct_dict

In [121]:
# get list of available pseudopotential files

pp_list = []

for filename in os.listdir("./simtool/pseudo/"):
    f = os.path.join("./simtool/pseudo/", filename)
    # get a list of all the PPs -- is this best instatiated here or globally?
    # if instanced here, the user could probably pass their own PPs to the constructor as well
    if os.path.isfile(f):
        pp_list.append(filename)

# filter by selected compound compositions
elements = np.unique([''.join([i for i in str(i.species) if i.isalpha()]) for i in struct.sites])
filtered_pp_list = [pp for pp in pp_list for e in elements if e in pp]

# make widgets for sim2l parameters

log = widgets.Select(
    options=["DEBUG", "INFO", "WARNING", "ERROR", "CRITICAL"],
    value='DEBUG',
    # rows=10,
    description='Log Level:',
)

walltime = widgets.Text(
    value='01:00:00',
    placeholder='walltime',
    description='walltime:',
    disabled=False
)
numnodes = widgets.IntText(
    value=8,
    placeholder='nodes',
    description='nodes:',
    disabled=False
) 
button = widgets.Button(
    description='run simtool',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='run to submit qe simtool'
)
ecutwfc = widgets.BoundedFloatText(
    value=50,
    min=50,
    max=400,
    step=10,
    description='ecutwfc:',
    disabled=False
)
kpoints = widgets.BoundedFloatText(
    value=6,
    min=1,
    max=20,
    step=1,
    description='kpoints:',
    disabled=False
)
ecutrho = widgets.BoundedFloatText(
    value=200,
    min=200,
    max=1600,
    step=40,
    description='ecutrho:',
    disabled=False
)
smearing = widgets.Select(
    options=['smearing','fixed'],
    value='fixed',
    rows = 2,
    description='smearing:',
    disabled=False
)
pp_menu1 = widgets.Combobox(
    placeholder="choose a pseudopotential",
    options=filtered_pp_list,
    description='pseudo 1:',
    disabled=False
)
pp_menu2 = widgets.Combobox(
    placeholder="choose a pseudopotential",
    options=filtered_pp_list,
    description='pseudo 2:',
    disabled=False
)

output = widgets.Output()

def runSim2l():
    inputs['loglevel'].value = log.value
    inputs['walltime'].value = walltime.value
    inputs['numnodes'].value = numnodes.value
    inputs['ecutwfc'].value = ecutwfc.value
    inputs['ecutrho'].value = ecutrho.value
    inputs['kpoints'].value = kpoints.value
    inputs['pps'].value = [pp_menu1.value, pp_menu2.value]
    inputs['smearing'].value = smearing.value
    inputs['struct_dict'].value = struct_dict

def on_button_clicked(b):
    with output:
        runSim2l()
        r = Run(simToolLocation,inputs)
        results = r.getResultSummary()
        print(r.read('spectra'))
    return results
        
        
results = button.on_click(on_button_clicked)

simulation = widgets.VBox([ecutrho, ecutwfc, kpoints, smearing, pp_menu1, pp_menu2])
run_details = widgets.VBox([walltime, numnodes, log])

accordion = widgets.VBox([widgets.Accordion(children=[simulation,run_details]),button,output])
display(accordion)

### Or, iterate through inputs directly

In [ ]:
ecutwfc_list = [50]
ecutrho_list = []
kpoints_list = [3, 6, 9, 12, 15]
struct_dict_list = []
results_list = []

inputs['pps'].value = ['O.pbe.upf', 'Zn.pbe.upf']
inputs['loglevel'].value = 'DEBUG'
inputs['smearing'].value = 'fixed'
inputs['struct_dict'].value = struct_dict

for ecutwfc in ecutwfc_list:
    for kpoints in kpoints_list:
        inputs['ecutwfc'].value = ecutwfc
        inputs['ecutrho'].value = 4*ecutwfc
        inputs['kpoints'].value = kpoints
        print("running sim with ",ecutwfc," and ",kpoints)
        r = Run(simToolLocation,inputs)
        results_list.append(r.getResultSummary())

running sim with  50  and  3
runname   = 29658a3a0b9a4f3aabe799f721cda155
outdir    = RUNS/29658a3a0b9a4f3aabe799f721cda155
cached    = False
submit --local papermill -f inputs.yaml
       /home/nanohub/nykiel.4/opt_prop/opt_prop_suite/simtool/relax_sim.ipynb
       relax_sim.ipynb


Input Notebook:  /home/nanohub/nykiel.4/opt_prop/opt_prop_suite/simtool/relax_sim.ipynb
Output Notebook: relax_sim.ipynb
Executing: 100%|██████████| 49/49 [14:26<00:00, 17.69s/cell]


running sim with  50  and  6
runname   = a843ea41768246d29acf50cb7ec4897e
outdir    = RUNS/a843ea41768246d29acf50cb7ec4897e
cached    = False
submit --local papermill -f inputs.yaml
       /home/nanohub/nykiel.4/opt_prop/opt_prop_suite/simtool/relax_sim.ipynb
       relax_sim.ipynb


Input Notebook:  /home/nanohub/nykiel.4/opt_prop/opt_prop_suite/simtool/relax_sim.ipynb
Output Notebook: relax_sim.ipynb
Executing:  73%|███████▎  | 36/49 [02:50<01:01,  4.73s/cell]
Traceback (most recent call last):
  File "/apps/share64/debian7/anaconda/anaconda-6/bin/papermill", line 10, in <module>
    sys.exit(papermill())
  File "/apps/share64/debian7/anaconda/anaconda-6/lib/python3.7/site-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/apps/share64/debian7/anaconda/anaconda-6/lib/python3.7/site-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/apps/share64/debian7/anaconda/anaconda-6/lib/python3.7/site-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/apps/share64/debian7/anaconda/anaconda-6/lib/python3.7/site-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/apps/share64/debian7/anaconda/anaconda-6/lib/python3

SimTool execution failed
The following outputs are missing: ['spectra', 'logreport']
running sim with  50  and  9
runname   = 5177d7e6067348f684ab608834168de2
outdir    = RUNS/5177d7e6067348f684ab608834168de2
cached    = False
submit --local papermill -f inputs.yaml
       /home/nanohub/nykiel.4/opt_prop/opt_prop_suite/simtool/relax_sim.ipynb
       relax_sim.ipynb


Input Notebook:  /home/nanohub/nykiel.4/opt_prop/opt_prop_suite/simtool/relax_sim.ipynb
Output Notebook: relax_sim.ipynb
Executing: 100%|██████████| 49/49 [1:40:14<00:00, 122.74s/cell]  
Traceback (most recent call last):
  File "/apps/share64/debian7/anaconda/anaconda-6/bin/papermill", line 10, in <module>
    sys.exit(papermill())
  File "/apps/share64/debian7/anaconda/anaconda-6/lib/python3.7/site-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/apps/share64/debian7/anaconda/anaconda-6/lib/python3.7/site-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/apps/share64/debian7/anaconda/anaconda-6/lib/python3.7/site-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/apps/share64/debian7/anaconda/anaconda-6/lib/python3.7/site-packages/click/core.py", line 610, in invoke
    return callback(*args, **kwargs)
  File "/apps/share64/debian7/anaconda/anaconda-6/lib/py

SimTool execution failed
The following outputs are missing: ['spectra']
running sim with  50  and  12
runname   = ab04e6b27d6a421ca3dd76e0191f1221
outdir    = RUNS/ab04e6b27d6a421ca3dd76e0191f1221
cached    = False
submit --local papermill -f inputs.yaml
       /home/nanohub/nykiel.4/opt_prop/opt_prop_suite/simtool/relax_sim.ipynb
       relax_sim.ipynb


Input Notebook:  /home/nanohub/nykiel.4/opt_prop/opt_prop_suite/simtool/relax_sim.ipynb
Output Notebook: relax_sim.ipynb
Executing:  78%|███████▊  | 38/49 [1:00:55<53:49, 293.63s/cell]

### Show User Predetermined Outputs and their Explainations

In [ ]:
outputs = getSimToolOutputs(simToolLocation)

In [ ]:
outputs

### Run simtool to obtain Predicted Raman Tensor and Spectrum Graph

In [ ]:
r.getResultSummary()

In [25]:
results.read('spectra')

AttributeError: 'NoneType' object has no attribute 'read'

In [ ]:
print(r.read('logreport'))

In [24]:
r.read('spectra')

NameError: name 'r' is not defined

In [116]:
#check inputs
r.input_dict

{'loglevel': 'DEBUG',
 'numnodes': 1,
 'walltime': '01:00:00',
 'pps': ['O.pbe.upf', 'Zn.pbe.upf'],
 'smearing': 'fixed',
 'ecutwfc': 50,
 'ecutrho': 200,
 'kpoints': 6,
 'epsil': True,
 'lraman': True,
 'struct_dict': {'@module': 'pymatgen.core.structure',
  '@class': 'Structure',
  'charge': None,
  'lattice': {'matrix': [[1.5920500003918698, -2.7575114888687695, 0.0],
    [1.5920500003918698, 2.7575114888687695, 0.0],
    [0.0, 0.0, 5.1551]],
   'a': 3.1841000007837397,
   'b': 3.1841000007837397,
   'c': 5.1551,
   'alpha': 90.0,
   'beta': 90.0,
   'gamma': 120.00000000000001,
   'volume': 45.26276950032406},
  'sites': [{'species': [{'element': 'Zn', 'occu': 1}],
    'abc': [0.6666666666666666, 0.3333333333333333, 0.5],
    'xyz': [1.5920500003918696, -0.9191704962895898, 2.57755],
    'label': 'Zn',
    'properties': {}},
   {'species': [{'element': 'Zn', 'occu': 1}],
    'abc': [0.3333333333333333, 0.6666666666666666, 0.0],
    'xyz': [1.5920500003918696, 0.9191704962895898, 0.

In [ ]:
#find output location
print(r.outdir)